# Programa Integral
**Campos Guevara Rogelio**

**201640376**

En primer lugar se importan las librerias.
Numpy se usará para el manejo de los arreglos y sus operaciones de manera sencilla.
De la librería math importaremos las funciones arcocoseno, coseno, y el valor de pi. 

In [ ]:
import numpy
from math import acos, cos, pi
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

Ahora, definimos el método integral. Dentro de este, se inicializa la variable integ, y se realiza la iteración en el rango n para el cálculo, sumando los resultados a la variable y regresando la misma al fin de la iteración.

In [ ]:
def integral(a_i, h, n):
    integ = 0.0
    for j in range(n):
        a_ij = a_i + (j + 0.5) * h
        integ += cos(a_ij) * h
    return integ

Ahora, declaramos e inicializamos los valores para a y b. 

In [ ]:
a = 0.0
b = pi / 2.0
my_int = numpy.zeros(1)
integral_sum = numpy.zeros(1)

Se inicializa el arreglo con valores dependiendo el núcleo que ingrese. En el caso del núcleo 0, se asigna el valor 100, en caso contrario el valor 0. 

In [ ]:
if rank == 0:
    n = numpy.full(1, 100, dtype=int)
else:
    n = numpy.zeros(1, dtype=int)

Se imprime el valor actual de n, se envia el valor de n en el núcleo 0 a todos los demás procesos y se vuelve a imprimir el valor en cada núcleo. En el caso del núcleo 0, los valores antes y después del broadcast serán 100, mientras que en los demás el valor antes del broadcast será de 0, y de 100 después de este.

In [ ]:
print("Process ", rank, " before n = ", n[0])
comm.Bcast(n, root=0)
print("Process ", rank, " after n = ", n[0])

Se define el valor de las particiones a partir del valor n, el número de núcleos y las variables definidas anteriormente.
Posteriormente se calcula el valor de a_i, y se hace el llamado a la función integral con los parámetros de cada núcleo.

In [ ]:
h = (b - a) / (n * size)
a_i = a + rank * h * n
my_int[0] = integral(a_i, h, n[0])

Ahora, el valor de la suma parcial realizada por cada núcleo se encuentra en la variable my_int, por lo que se imprime su valor y se envía a la raíz, sumandose al valor final.

In [ ]:
print("Process ", rank, " has the partial integral ", my_int[0])
comm.Reduce(my_int, integral_sum, MPI.SUM, 0)

Únicamente el núcleo 0 imprime el valor final de la suma.

In [ ]:
# Only print the result in process 0
if rank == 0:
    print('The Integral Sum =', integral_sum[0])

El resultado de la ejecución del programa se muestra a continuación:

In [ ]:
Process  1  before n =  0
Process  1  after n =  100
Process  1  has the partial integral  0.2928939716117343
Process  0  before n =  100
Process  0  after n =  100
Process  0  has the partial integral  0.7071085986023695
The Integral Sum = 1.0000025702141038